#1 Импорт необходимых библиотек и загрузка данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from google.colab import files

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 30)
pd.set_option("display.precision", 4)
pd.set_option("plotting.backend", "matplotlib")

In [ ]:
uploaded = files.upload()

!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c urfuaieng2022clf2

!unzip urfuaieng2022clf2.zip

mv: cannot stat 'kaggle.json': No such file or directory
urfuaieng2022clf2.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  urfuaieng2022clf2.zip
  inflating: Submission_clf.csv      
  inflating: Test_Who.csv            
  inflating: Train_Target_clf.csv    
  inflating: Train_Who.csv           
  inflating: assessments.csv         
  inflating: courses.csv             
  inflating: studentInfo.csv         
  inflating: studentRegistration.csv  
  inflating: studentVle.csv          
  inflating: vle.csv                 


In [ ]:
Train_Who = pd.read_csv('Train_Who.csv')
Train_Target_clf = pd.read_csv('Train_Target_clf.csv')
Test_Who = pd.read_csv('Test_Who.csv')
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
studentInfo = pd.read_csv('studentInfo.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')
studentVle = pd.read_csv('studentVle.csv')
vle = pd.read_csv('vle.csv')

#2 EDA

##2.1 Базовое описание данных

Дополнительные датасеты:

###2.1.1 `assessments`

In [ ]:
print(assessments.info())
assessments.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        206 non-null    object 
 1   code_presentation  206 non-null    object 
 2   id_assessment      206 non-null    int64  
 3   assessment_type    206 non-null    object 
 4   date               195 non-null    float64
 5   weight             206 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.8+ KB
None


,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


Этот файл содержит информацию об оценках в презентациях модулей. Обычно за каждой презентацией следует ряд оценок, за которыми следует итоговый экзамен. CSV содержит столбцы:
  * code_module – идентификационный код модуля, к которому относится оценка.
  * code_presentation - идентификационный код презентации, к которой относится оценка.
  * id_assessment – идентификационный номер оценки.
  * assessment_type – тип оценки. Существует три типа оценок: оценка с оценкой преподавателя (TMA), компьютерная оценка (CMA) и итоговый экзамен (Exam).
  * date – информация об окончательной дате подачи оценки, рассчитываемая как количество дней с начала презентации модуля. Дата начала презентации имеет номер 0 (ноль).
  * weight - вес оценки в %. Как правило, экзамены рассматриваются отдельно и имеют вес 100%; сумма всех других оценок равна 100%.

Если информация о дате итогового экзамена отсутствует, это относится к концу последней недели презентаций.

###2.1.2 `courses`

In [ ]:
print(courses.info())
courses.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   code_module                 22 non-null     object
 1   code_presentation           22 non-null     object
 2   module_presentation_length  22 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 656.0+ bytes
None


,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


Файл содержит список всех доступных модулей и их презентаций. В колонках представлены:
  * code_module – кодовое название модуля, которое служит идентификатором.
  * code_presentation – кодовое название презентации. Она состоит из года и “B” для презентации, начинающейся в феврале, и “J” для презентации, начинающейся в октябре.
  * module_presentation_length   - продолжительность модуля-презентации в днях.

Структура презентаций B и J может отличаться, и поэтому хорошей практикой является анализ презентаций B и J по отдельности. Тем не менее, для некоторых презентаций соответствующая предыдущая презентация B / J не существует, и поэтому презентация J должна использоваться для информирования презентации B, или наоборот. В наборе данных это относится к модулям CCC, EEE и GGG.

###2.1.3 `studentInfo`

In [ ]:
print(studentInfo.info())
studentInfo.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           32593 non-null  object
 1   code_presentation     32593 non-null  object
 2   id_student            32593 non-null  int64 
 3   gender                32593 non-null  object
 4   region                32593 non-null  object
 5   highest_education     32593 non-null  object
 6   imd_band              31482 non-null  object
 7   age_band              32593 non-null  object
 8   num_of_prev_attempts  32593 non-null  int64 
 9   studied_credits       32593 non-null  int64 
 10  disability            32593 non-null  object
dtypes: int64(3), object(8)
memory usage: 2.7+ MB
None


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N


Этот файл содержит демографическую информацию о студентах вместе с их результатами. Файл содержит следующие столбцы:
  * code_module – идентификационный код для модуля, на котором зарегистрирован студент.
  * code_presentation - идентификационный код презентации, во время которой студент регистрируется в модуле.
  * id_student – уникальный идентификационный номер студента.
  * gender – пол учащегося.
  * region – определяет географический регион, в котором студент проживал во время прохождения модуля-презентации.
  * highest_education – наивысший уровень образования учащихся при входе на презентацию модуля.
  * imd_band – задает индекс множественной деградации диапазона места, где студент жил во время презентации модуля.
  * age_band – возрастная группа учащегося.
  * num_of_prev_attempts – количество попыток студента выполнить этот модуль.
  * studied_credits – общее количество кредитов по модулям, которые студент изучает в данный момент.
  * disability – указывает, заявил ли учащийся о своей инвалидности.
  * final_result – итоговый результат студента в презентации модуля.

###2.1.4 `studentRegistration`

In [ ]:
print(studentRegistration.info())
studentRegistration.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        32593 non-null  object 
 1   code_presentation  32593 non-null  object 
 2   id_student         32593 non-null  int64  
 3   date_registration  32548 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1018.7+ KB
None


,code_module,code_presentation,id_student,date_registration
0,AAA,2013J,11391,-159.0
1,AAA,2013J,28400,-53.0
2,AAA,2013J,30268,-92.0
3,AAA,2013J,31604,-52.0
4,AAA,2013J,32885,-176.0


Этот файл содержит информацию о времени, когда студент зарегистрировался для участия в презентации модуля. Для студентов, которые не зарегистрировались, также записывается дата отмены регистрации. Файл содержит пять столбцов:
  * code_module – идентификационный код для модуля.
  * code_presentation - идентификационный код презентации.
  * id_student – уникальный идентификационный номер студента.
  * date_registration – дата регистрации студента на презентации модуля, это количество дней, измеренное относительно начала презентации модуля (например, отрицательное значение -30 означает, что студент зарегистрировался на презентации модуля за 30 дней до ее начала).
  * date_unregistration – дата отмены регистрации студента с презентации модуля, это количество дней, измеренное относительно начала презентации модуля. У студентов, окончивших курс, это поле пустое. Студенты, которые не зарегистрировались, могут отказаться от участия в качестве значения столбца final_result в файле StudentInfo.csv. **(отстутствует)**

###2.1.5 `studentVle`

In [ ]:
print(studentVle.info())
studentVle.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10655280 entries, 0 to 10655279
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code_module        object
 1   code_presentation  object
 2   id_student         int64 
 3   id_site            int64 
 4   date               int64 
 5   sum_click          int64 
dtypes: int64(4), object(2)
memory usage: 487.8+ MB
None


,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


Файл studentVle.csv содержит информацию о взаимодействии каждого студента с материалами VLE. Этот файл содержит следующие столбцы:
  * code_module – идентификационный код для модуля.
  * code_presentation - идентификационный код презентации модуля.
  * id_student – уникальный идентификационный номер студента.
  * id_site - идентификационный номер для материала VLE.
  * date – дата взаимодействия студента с материалом, измеряемая как количество дней с начала презентации модуля.
  * sum_click – количество раз, когда студент взаимодействует с материалом за этот день.

###2.1.6 `vle`

In [ ]:
print(vle.info())
vle.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6364 entries, 0 to 6363
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_site            6364 non-null   int64  
 1   code_module        6364 non-null   object 
 2   code_presentation  6364 non-null   object 
 3   activity_type      6364 non-null   object 
 4   week_from          1121 non-null   float64
 5   week_to            1121 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 298.4+ KB
None


,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,NaN,NaN
1,546712,AAA,2013J,oucontent,NaN,NaN
2,546998,AAA,2013J,resource,NaN,NaN
3,546888,AAA,2013J,url,NaN,NaN
4,547035,AAA,2013J,resource,NaN,NaN


Файл csv содержит информацию о доступных материалах в VLE. Обычно это html-страницы, PDF-файлы и т.д. Учащиеся имеют доступ к этим материалам онлайн, и их взаимодействие с материалами записывается. Файл vle.csv содержит следующие столбцы:
  * id_site – идентификационный номер материала.
  * code_module – идентификационный код для модуля.
  * code_presentation - идентификационный код презентации.
  * activity_type – роль, связанная с материалом модуля.
  * week_from – неделя, с которой планируется использовать материал.
  * week_to – неделя, до которой планируется использовать материал.

###2.1.7 `Train_Who`

Основной train датасет:

In [ ]:
print(Train_Who.info())
Train_Who.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26074 entries, 0 to 26073
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 26074 non-null  int64 
 1   id_student         26074 non-null  int64 
 2   code_module        26074 non-null  object
 3   code_presentation  26074 non-null  object
dtypes: int64(2), object(2)
memory usage: 814.9+ KB
None


,ID,id_student,code_module,code_presentation
0,0,464685,DDD,2014B
1,1,606593,DDD,2013J
2,2,1674799,CCC,2014J
3,3,195246,FFF,2014J
4,4,582080,FFF,2013J


###2.1.8 `Train_Target_clf`

train с целевой метрикой:

In [ ]:
print(Train_Target_clf.info())
Train_Target_clf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26074 entries, 0 to 26073
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   ID            26074 non-null  int64
 1   final_result  26074 non-null  int64
dtypes: int64(2)
memory usage: 407.5 KB
None


,ID,final_result
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0


###2.1.9 `Test_Who`

Основной test датасет:

In [ ]:
print(Test_Who.info())
Test_Who.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6519 entries, 0 to 6518
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 6519 non-null   int64 
 1   id_student         6519 non-null   int64 
 2   code_module        6519 non-null   object
 3   code_presentation  6519 non-null   object
dtypes: int64(2), object(2)
memory usage: 203.8+ KB
None


,ID,id_student,code_module,code_presentation
0,26074,2588655,DDD,2014J
1,26075,543628,CCC,2014B
2,26076,387276,BBB,2013B
3,26077,692219,CCC,2014J
4,26078,516212,BBB,2013J


##2.2 Преобразованые таблиц и данных

###2.2.1 Основа train датасета

Прежде всего создадим основу для тренировочного датасета. Для этого сначала сджойним таблицы Train_Who и Train_Target_clf:

In [ ]:
train_base = pd.merge(left=Train_Who, right=Train_Target_clf, how='left', on='ID')
train_base.sample(5)

,ID,id_student,code_module,code_presentation,final_result
280,280,696410,FFF,2014J,0
8942,8942,155735,BBB,2013B,0
12560,12560,597010,GGG,2014B,1
17325,17325,643758,CCC,2014J,0
12010,12010,526060,FFF,2013B,0


###2.2.2 Данные по материалам курса

Дальше посмотрим на таблицу studentVle. Она содержит огромное количество строк, прежде чем джойнить ее к другим данным, необходимо уменьшить ее размерность. Ее размеры:

In [ ]:
studentVle.shape

(10655280, 6)

In [ ]:
studentVle.sample(5)

,code_module,code_presentation,id_student,id_site,date,sum_click
4504223,DDD,2014B,626723,772729,47,1
9377211,FFF,2014J,646074,883081,30,6
9495630,FFF,2014J,690296,882545,50,4
4114784,DDD,2013J,575192,673533,134,1
4444192,DDD,2014B,2080340,772995,22,2


В сущности она отражает количество внимания, которое студент уделил учебным материалам к презентации. По логике, большое внимание должно положительно сказываться на оценке студента.

Рассмотрим прмер случайного студента:

In [ ]:
exemple = studentVle[studentVle.id_student.isin([623164])]
exemple.sample(5)

,code_module,code_presentation,id_student,id_site,date,sum_click
4610025,DDD,2014B,623164,773008,125,2
4560441,DDD,2014B,623164,772733,84,2
4397985,DDD,2014B,623164,772705,5,4
4543847,DDD,2014B,623164,772729,71,1
4455403,DDD,2014B,623164,773312,24,1


In [ ]:
exemple.sum_click.unique()

array([ 1,  7,  2,  5,  4,  9,  3, 11,  6, 10, 12,  8, 17, 14])

По логике, распределение количества кликов по дням большого смысла не имеет, потому что студенты могут проходить задания в произвольном темпе. Поэтому можно агрегировать количество кликов, устранив распределение по дням, без потери смысла данных.

Также можно приджойнить, таблицу vle, где отражены соответствующий тип ID учебного материала. Имеет смысл сохранить разделение по типам материалов, потому что, возможно, внимаение к одним материалам приносит больше пользы, чем внимание к другим.

Также в таблице vle для начала посчитаем количество материлов каждого типа в презентации, чтобы в дальнейшим рассчитать отношение количества просмотренных студентом материлов к вообще доступным. Очевидно, чем больше просмотрено, тем лучше должна быть оценка. Попробуем:

In [ ]:
vle_add = vle.groupby(['code_module', 'code_presentation', 'activity_type'], as_index=False).size().rename(columns={'size': 'num_mat_in_type'})

vle_add.sample(5)

,code_module,code_presentation,activity_type,num_mat_in_type
74,CCC,2014J,subpage,31
42,BBB,2014B,oucontent,3
122,EEE,2013J,ouwiki,2
117,EEE,2013J,dualpane,1
101,DDD,2014B,oucollaborate,5


Присоединим к таблице studentVle информацию о типе контента из таблицы vle:

In [ ]:
id_site_clicks_with_types = pd.merge(left=studentVle, right=vle[['code_module', 'code_presentation', 'id_site', 'activity_type']]
                                     , on=['code_module', 'code_presentation', 'id_site'], how='left')

id_site_clicks_with_types[id_site_clicks_with_types.id_student == 243657]

,code_module,code_presentation,id_student,id_site,date,sum_click,activity_type
6253624,FFF,2013B,243657,527219,-18,1,subpage
6253626,FFF,2013B,243657,527439,-18,1,resource
6253627,FFF,2013B,243657,526819,-18,1,oucontent
6253628,FFF,2013B,243657,527204,-18,1,subpage
6253629,FFF,2013B,243657,526820,-18,8,oucontent
...,...,...,...,...,...,...,...
7198520,FFF,2013B,243657,527219,234,2,subpage
7198521,FFF,2013B,243657,526737,234,2,forumng
7200040,FFF,2013B,243657,527231,240,2,subpage
7200041,FFF,2013B,243657,526721,240,4,homepage


Уберем даты из рассмотрния, просуммируем клики:

In [ ]:
site_clicks_by_id = id_site_clicks_with_types.groupby(['code_module', 'code_presentation', 'id_student', 'id_site', 'activity_type'],
                                                      as_index=False).sum_click.sum()

site_clicks_by_id[site_clicks_by_id.id_student == 243657]

,code_module,code_presentation,id_student,id_site,activity_type,sum_click
1138782,FFF,2013B,243657,526721,homepage,1031
1138783,FFF,2013B,243657,526735,forumng,18
1138784,FFF,2013B,243657,526737,forumng,441
1138785,FFF,2013B,243657,526738,forumng,441
1138786,FFF,2013B,243657,526741,oucontent,110
...,...,...,...,...,...,...
1138986,FFF,2013B,243657,527507,ouelluminate,8
1138987,FFF,2013B,243657,527518,ouelluminate,8
1138988,FFF,2013B,243657,527524,dataplus,6
1138989,FFF,2013B,243657,527526,dataplus,5


Теперь посчитаем количество контента по типам, которые просматиривал студент, при этом количество кликов также суммируем:

In [ ]:
site_clicks_by_id['num_seen_may_in_act_type'] = site_clicks_by_id['activity_type']

count_seen_sites = site_clicks_by_id.groupby(['code_module', 'code_presentation', 'id_student', 'activity_type'],
                                             as_index=False).agg({'num_seen_may_in_act_type': np.size, 'sum_click': np.sum})

count_seen_sites[count_seen_sites.id_student == 243657]

,code_module,code_presentation,id_student,activity_type,num_seen_may_in_act_type,sum_click
151048,FFF,2013B,243657,dataplus,3,14
151049,FFF,2013B,243657,dualpane,5,10
151050,FFF,2013B,243657,forumng,3,900
151051,FFF,2013B,243657,homepage,1,1031
151052,FFF,2013B,243657,oucontent,81,2269
151053,FFF,2013B,243657,ouelluminate,2,16
151054,FFF,2013B,243657,ouwiki,3,42
151055,FFF,2013B,243657,page,11,13
151056,FFF,2013B,243657,questionnaire,6,16
151057,FFF,2013B,243657,quiz,9,792


Сджойним получившуюся таблицу с vle_add, чтобы добавить количество доступного контента по типам, и рассчитаем искомое отношение:

In [ ]:
site_interaction = pd.merge(left=count_seen_sites, right=vle_add, on=['code_module', 'code_presentation', 'activity_type'])

site_interaction['mat_seen_ratio'] = site_interaction.num_seen_may_in_act_type / site_interaction.num_mat_in_type
site_interaction = site_interaction[['code_module', 'code_presentation', 'id_student', 'activity_type', 'mat_seen_ratio', 'sum_click']]

site_interaction[site_interaction.id_student == 243657]

,code_module,code_presentation,id_student,activity_type,mat_seen_ratio,sum_click
149765,FFF,2013B,243657,dataplus,0.6000,14
150369,FFF,2013B,243657,dualpane,0.8333,10
151295,FFF,2013B,243657,forumng,0.6000,900
152776,FFF,2013B,243657,homepage,1.0000,1031
154285,FFF,2013B,243657,oucontent,0.7500,2269
155742,FFF,2013B,243657,ouelluminate,0.1250,16
156962,FFF,2013B,243657,ouwiki,0.5000,42
158201,FFF,2013B,243657,page,0.4231,13
159599,FFF,2013B,243657,questionnaire,0.4286,16
160538,FFF,2013B,243657,quiz,0.8182,792


Посмотрим на виды активности:

In [ ]:
site_interaction.activity_type.unique()

array(['forumng', 'homepage', 'oucontent', 'resource', 'subpage', 'url',
       'dataplus', 'glossary', 'oucollaborate', 'quiz', 'ouelluminate',
       'sharedsubpage', 'questionnaire', 'page', 'externalquiz', 'ouwiki',
       'dualpane', 'folder', 'repeatactivity', 'htmlactivity'],
      dtype=object)

Для корректного представления информации по видам активностей нужно преобразовать связку "тип-отношение/количество кликов" к другому виду:

In [ ]:
names_dict = {'mat_seen_ratiodataplus': 'mat_seen_ratio_dataplus',
              'mat_seen_ratiodualpane': 'mat_seen_ratio_dualpane',
              'mat_seen_ratioexternalquiz': 'mat_seen_ratio_externalquiz',
              'mat_seen_ratiofolder': 'mat_seen_ratio_folder',
              'mat_seen_ratioforumng': 'mat_seen_ratio_forumng',
              'mat_seen_ratioglossary': 'mat_seen_ratio_glossary',
              'mat_seen_ratiohomepage': 'mat_seen_ratio_homepage',
              'mat_seen_ratiohtmlactivity': 'mat_seen_ratio_htmlactivity',
              'mat_seen_ratiooucollaborate': 'mat_seen_ratio_oucollaborate',
              'mat_seen_ratiooucontent': 'mat_seen_ratio_oucontent',
              'mat_seen_ratioouelluminate': 'mat_seen_ratio_ouelluminate',
              'mat_seen_ratioouwiki': 'mat_seen_ratio_ouwiki',
              'mat_seen_ratiopage': 'mat_seen_ratio_page',
              'mat_seen_ratioquestionnaire': 'mat_seen_ratio_questionnaire',
              'mat_seen_ratioquiz': 'mat_seen_ratio_quiz',
              'mat_seen_ratioresource': 'mat_seen_ratio_resource',
              'mat_seen_ratiosubpage': 'mat_seen_ratio_subpage',
              'mat_seen_ratiourl': 'mat_seen_ratio_url',
              'mat_seen_ratiosharedsubpage': 'mat_seen_ratio_sharedsubpage',
              'mat_seen_ratiorepeatactivity': 'mat_seen_ratio_repeatactivity',
              'sum_clickdataplus': 'sum_click_dataplus',
              'sum_clickdualpane': 'sum_click_dualpane',
              'sum_clickexternalquiz': 'sum_click_externalquiz',
              'sum_clickfolder': 'sum_click_folder',
              'sum_clickforumng': 'sum_click_forumng',
              'sum_clickglossary': 'sum_click_glossary',
              'sum_clickhomepage': 'sum_click_homepage',
              'sum_clickhtmlactivity': 'sum_click_htmlactivity',
              'sum_clickoucollaborate': 'sum_click_oucollaborate',
              'sum_clickoucontent': 'sum_click_oucontent',
              'sum_clickouelluminate': 'sum_click_ouelluminate',
              'sum_clickouwiki': 'sum_click_ouwiki',
              'sum_clickpage': 'sum_click_page',
              'sum_clickquestionnaire': 'sum_click_questionnaire',
              'sum_clickquiz': 'sum_click_quiz',
              'sum_clickresource': 'sum_click_resource',
              'sum_clicksubpage': 'sum_click_subpage',
              'sum_clickurl': 'sum_click_url',
              'sum_clicksharedsubpage': 'sum_click_sharedsubpage',
              'sum_clickrepeatactivity': 'sum_click_repeatactivity'}

In [ ]:
new_columns_dict = ['mat_seen_ratio_dataplus', 'mat_seen_ratio_dualpane',
                    'mat_seen_ratio_externalquiz', 'mat_seen_ratio_folder',
                    'mat_seen_ratio_forumng', 'mat_seen_ratio_glossary',
                    'mat_seen_ratio_homepage', 'mat_seen_ratio_htmlactivity',
                    'mat_seen_ratio_oucollaborate', 'mat_seen_ratio_oucontent',
                    'mat_seen_ratio_ouelluminate', 'mat_seen_ratio_ouwiki',
                    'mat_seen_ratio_page', 'mat_seen_ratio_questionnaire',
                    'mat_seen_ratio_quiz', 'mat_seen_ratio_resource',
                    'mat_seen_ratio_subpage', 'mat_seen_ratio_url',
                    'mat_seen_ratio_sharedsubpage', 'mat_seen_ratio_repeatactivity',
                    'sum_click_dataplus', 'sum_click_dualpane',
                    'sum_click_externalquiz', 'sum_click_folder',
                    'sum_click_forumng', 'sum_click_glossary',
                    'sum_click_homepage', 'sum_click_htmlactivity',
                    'sum_click_oucollaborate', 'sum_click_oucontent',
                    'sum_click_ouelluminate', 'sum_click_ouwiki',
                    'sum_click_page', 'sum_click_questionnaire',
                    'sum_click_quiz', 'sum_click_resource',
                    'sum_click_subpage', 'sum_click_url',
                    'sum_click_sharedsubpage', 'sum_click_repeatactivity']

In [ ]:
pivot_table = pd.pivot(data=site_interaction, columns='activity_type', values=['mat_seen_ratio', 'sum_click'])

pivot_table.columns = pivot_table.columns.to_flat_index().str.join('')
pivot_table = pivot_table.rename(columns=names_dict)

buf_table = pd.merge(left=site_interaction, right=pivot_table, left_index=True, right_index=True)

stud_site_interaction = buf_table.groupby(['code_module', 'code_presentation', 'id_student'],
                                          as_index=False)[new_columns_dict].sum()

stud_site_interaction[stud_site_interaction.id_student == 243657]

,code_module,code_presentation,id_student,mat_seen_ratio_dataplus,mat_seen_ratio_dualpane,mat_seen_ratio_externalquiz,mat_seen_ratio_folder,mat_seen_ratio_forumng,mat_seen_ratio_glossary,mat_seen_ratio_homepage,mat_seen_ratio_htmlactivity,mat_seen_ratio_oucollaborate,mat_seen_ratio_oucontent,mat_seen_ratio_ouelluminate,mat_seen_ratio_ouwiki,...,sum_click_glossary,sum_click_homepage,sum_click_htmlactivity,sum_click_oucollaborate,sum_click_oucontent,sum_click_ouelluminate,sum_click_ouwiki,sum_click_page,sum_click_questionnaire,sum_click_quiz,sum_click_resource,sum_click_subpage,sum_click_url,sum_click_sharedsubpage,sum_click_repeatactivity
19893,FFF,2013B,243657,0.6,0.8333,0.0,0.0,0.6,0.0,1.0,0.0,0.0,0.75,0.125,0.5,...,0.0,1031.0,0.0,0.0,2269.0,16.0,42.0,13.0,16.0,792.0,43.0,579.0,65.0,0.0,0.0


Размеры получившейся таблицы:

In [ ]:
stud_site_interaction.shape

(29228, 43)

###2.2.4 Информация о студентах

Обогатим информацию о пользователях из studentInfo, добавив туда информацю о дате регистрации из studentRegistration:

In [ ]:
students_info = pd.merge(left=studentInfo, right=studentRegistration
                          , on=['code_module', 'code_presentation', 'id_student'], how='left')
students_info.info()
students_info.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32593 entries, 0 to 32592
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   code_module           32593 non-null  object 
 1   code_presentation     32593 non-null  object 
 2   id_student            32593 non-null  int64  
 3   gender                32593 non-null  object 
 4   region                32593 non-null  object 
 5   highest_education     32593 non-null  object 
 6   imd_band              31482 non-null  object 
 7   age_band              32593 non-null  object 
 8   num_of_prev_attempts  32593 non-null  int64  
 9   studied_credits       32593 non-null  int64  
 10  disability            32593 non-null  object 
 11  date_registration     32548 non-null  float64
dtypes: float64(1), int64(3), object(8)
memory usage: 3.2+ MB


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,date_registration
29985,FFF,2014J,2427845,M,South East Region,HE Qualification,10-20,35-55,0,60,N,-46.0
9924,CCC,2014B,613501,M,East Anglian Region,Lower Than A Level,70-80%,0-35,0,60,N,-134.0
31646,GGG,2014B,630169,F,North Western Region,Lower Than A Level,0-10%,35-55,0,30,Y,-8.0
25510,FFF,2013J,595769,M,North Western Region,HE Qualification,20-30%,0-35,0,120,N,-53.0
1972,BBB,2013B,549352,F,North Western Region,Lower Than A Level,10-20,0-35,0,60,N,-78.0


Кое-где не хватает даты регистрации. Посмотрим на эти данные:

In [ ]:
students_info[students_info.date_registration.isna()].sample(5)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,date_registration
11249,CCC,2014J,544271,M,South Region,A Level or Equivalent,60-70%,0-35,0,90,N,NaN
16092,DDD,2013J,835603,M,North Western Region,HE Qualification,20-30%,55<=,0,150,N,NaN
18249,DDD,2014J,604632,M,South West Region,A Level or Equivalent,0-10%,0-35,0,180,N,NaN
17559,DDD,2014B,2710343,M,North Western Region,Lower Than A Level,0-10%,0-35,1,60,N,NaN
14876,DDD,2013J,470119,M,West Midlands Region,Lower Than A Level,40-50%,0-35,1,240,N,NaN


С виду обычные данные. Посмотрим, влиеят ли как-то дата регистрации на итоговую оценку:

In [ ]:
pd.merge(left=students_info, right=train_base, on=['code_module', 'code_presentation', 'id_student'],
         how='right')[['date_registration', 'final_result']].corr()

,date_registration,final_result
date_registration,1.0000,0.0519
final_result,0.0519,1.0000


Нет, не влияет. Удалим этот столбец.

In [ ]:
students_info = students_info.drop(columns='date_registration')

Также нехватает данных imd_dand. Судя по описанию к данным это индекс, отражающий качество жизни человека. Чем выше, тем лучше. Зависит от социальных показателей человека, района проживания, образования и тд.

In [ ]:
students_info.imd_band.value_counts()

20-30%     3654
30-40%     3539
10-20      3516
0-10%      3311
40-50%     3256
50-60%     3124
60-70%     2905
70-80%     2879
80-90%     2762
90-100%    2536
Name: imd_band, dtype: int64

Посмотрим:

In [ ]:
students_info[students_info.imd_band.isna()].sample(5)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability
12586,CCC,2014J,683668,F,Ireland,Post Graduate Qualification,NaN,0-35,0,90,N
12312,CCC,2014J,650271,M,North Region,HE Qualification,NaN,35-55,0,120,N
12967,CCC,2014J,2179304,F,North Region,Lower Than A Level,NaN,35-55,0,90,N
13550,DDD,2013B,477160,F,Ireland,HE Qualification,NaN,35-55,1,120,N
5163,BBB,2014B,537171,F,North Region,No Formal quals,NaN,0-35,0,90,N


В таких случаях найдем самую распространенную категорию по похожим характеристикам. Если данных для абсолютно похожих людей нет, то будем ослаблять условия:

In [ ]:
def find_imd(data):

  for id in data[data.imd_band.isna()].id_student:
    meta_condition = data.id_student == id

    gender = data.gender == data[meta_condition].gender.values[0]
    region = data.region == data[meta_condition].region.values[0]
    education = data.highest_education == data[meta_condition].highest_education.values[0]
    age = data.age_band == data[meta_condition].age_band.values[0]
    disability = data.disability == data[meta_condition].disability.values[0]

    imd_rate = data[disability & region & age & education & gender][['imd_band']].value_counts()

    if len(imd_rate) == 0:
      imd_rate = data[disability & region & age & education][['imd_band']].value_counts()
      if len(imd_rate) == 0:
        imd_rate = data[disability & region & age][['imd_band']].value_counts()
        if len(imd_rate) == 0:
          imd_rate = data[disability & region][['imd_band']].value_counts()
          if len(imd_rate) == 0:
            imd_rate = data[disability][['imd_band']].value_counts()
            imd = imd_rate.index[0][0]
          else:
            imd = imd_rate.index[0][0]
        else:
          imd = imd_rate.index[0][0]
      else:
        imd = imd_rate.index[0][0]
    else:
      imd = imd_rate.index[0][0]

    data.imd_band = data.imd_band.mask(meta_condition, imd)

  return data

In [ ]:
students_info = find_imd(students_info)

In [ ]:
students_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32593 entries, 0 to 32592
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           32593 non-null  object
 1   code_presentation     32593 non-null  object
 2   id_student            32593 non-null  int64 
 3   gender                32593 non-null  object
 4   region                32593 non-null  object
 5   highest_education     32593 non-null  object
 6   imd_band              32593 non-null  object
 7   age_band              32593 non-null  object
 8   num_of_prev_attempts  32593 non-null  int64 
 9   studied_credits       32593 non-null  int64 
 10  disability            32593 non-null  object
dtypes: int64(3), object(8)
memory usage: 3.0+ MB


##2.3 Джойн таблиц

train датасет:

In [ ]:
train_stud = pd.merge(left=train_base, right=students_info,
                      on=['code_module', 'code_presentation', 'id_student'],
                      how='left')

train = pd.merge(left=train_stud, right=stud_site_interaction,
                 on=['code_module', 'code_presentation', 'id_student'],
                 how='left')

print(train.info())
train.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26074 entries, 0 to 26073
Data columns (total 53 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ID                             26074 non-null  int64  
 1   id_student                     26074 non-null  int64  
 2   code_module                    26074 non-null  object 
 3   code_presentation              26074 non-null  object 
 4   final_result                   26074 non-null  int64  
 5   gender                         26074 non-null  object 
 6   region                         26074 non-null  object 
 7   highest_education              26074 non-null  object 
 8   imd_band                       26074 non-null  object 
 9   age_band                       26074 non-null  object 
 10  num_of_prev_attempts           26074 non-null  int64  
 11  studied_credits                26074 non-null  int64  
 12  disability                     26074 non-null 

,ID,id_student,code_module,code_presentation,final_result,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,mat_seen_ratio_dataplus,mat_seen_ratio_dualpane,...,sum_click_glossary,sum_click_homepage,sum_click_htmlactivity,sum_click_oucollaborate,sum_click_oucontent,sum_click_ouelluminate,sum_click_ouwiki,sum_click_page,sum_click_questionnaire,sum_click_quiz,sum_click_resource,sum_click_subpage,sum_click_url,sum_click_sharedsubpage,sum_click_repeatactivity
8952,8952,694807,CCC,2014J,0,M,South Region,HE Qualification,90-100%,0-35,0,60,N,0.0,0.0,...,0.0,28.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,95.0,8.0,15.0,2.0,0.0,0.0
19203,19203,609265,DDD,2014B,1,F,East Midlands Region,Lower Than A Level,0-10%,0-35,0,60,Y,0.0,0.0,...,6.0,692.0,0.0,14.0,134.0,0.0,51.0,0.0,0.0,0.0,102.0,502.0,149.0,0.0,0.0
16773,16773,530270,CCC,2014B,0,M,South Region,A Level or Equivalent,50-60%,0-35,0,150,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13697,13697,633960,DDD,2014B,1,M,North Region,Post Graduate Qualification,0-10%,35-55,0,60,N,0.0,0.0,...,0.0,373.0,0.0,148.0,176.0,0.0,190.0,0.0,0.0,0.0,146.0,524.0,32.0,0.0,0.0
20957,20957,322137,DDD,2014J,1,M,London Region,HE Qualification,30-40%,55<=,0,60,N,0.0,0.0,...,86.0,651.0,0.0,74.0,99.0,0.0,0.0,0.0,0.0,0.0,242.0,416.0,65.0,0.0,0.0


test датасет:

In [ ]:
test_stud = pd.merge(left=Test_Who, right=students_info,
                     on=['code_module', 'code_presentation', 'id_student'],
                     how='left')

test = pd.merge(left=test_stud, right=stud_site_interaction,
                on=['code_module', 'code_presentation', 'id_student'],
                how='left')

print(test.info())
test.sample(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6519 entries, 0 to 6518
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ID                             6519 non-null   int64  
 1   id_student                     6519 non-null   int64  
 2   code_module                    6519 non-null   object 
 3   code_presentation              6519 non-null   object 
 4   gender                         6519 non-null   object 
 5   region                         6519 non-null   object 
 6   highest_education              6519 non-null   object 
 7   imd_band                       6519 non-null   object 
 8   age_band                       6519 non-null   object 
 9   num_of_prev_attempts           6519 non-null   int64  
 10  studied_credits                6519 non-null   int64  
 11  disability                     6519 non-null   object 
 12  mat_seen_ratio_dataplus        5864 non-null   f

,ID,id_student,code_module,code_presentation,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,mat_seen_ratio_dataplus,mat_seen_ratio_dualpane,mat_seen_ratio_externalquiz,...,sum_click_glossary,sum_click_homepage,sum_click_htmlactivity,sum_click_oucollaborate,sum_click_oucontent,sum_click_ouelluminate,sum_click_ouwiki,sum_click_page,sum_click_questionnaire,sum_click_quiz,sum_click_resource,sum_click_subpage,sum_click_url,sum_click_sharedsubpage,sum_click_repeatactivity
2265,28339,2689536,DDD,2013B,M,Scotland,HE Qualification,40-50%,35-55,0,120,Y,0.0,0.0,0.4286,...,2.0,189.0,0.0,0.0,141.0,13.0,25.0,0.0,0.0,0.0,60.0,202.0,33.0,0.0,0.0
4170,30244,355510,DDD,2013J,F,North Western Region,A Level or Equivalent,90-100%,35-55,1,150,N,0.0,0.0,1.0000,...,456.0,696.0,0.0,42.0,152.0,0.0,164.0,0.0,0.0,0.0,114.0,528.0,110.0,0.0,0.0
4444,30518,88340,BBB,2014B,F,Wales,Lower Than A Level,30-40%,0-35,0,60,Y,0.0,0.0,0.0000,...,0.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,38.0,3.0,20.0,0.0,0.0
6309,32383,559382,BBB,2013J,F,South West Region,A Level or Equivalent,80-90%,0-35,0,180,N,0.0,0.0,0.0000,...,0.0,324.0,0.0,1.0,60.0,0.0,0.0,0.0,0.0,78.0,25.0,66.0,93.0,0.0,0.0
6276,32350,491926,DDD,2013J,M,Wales,Lower Than A Level,10-20,35-55,0,60,Y,0.0,0.0,1.0000,...,37.0,779.0,0.0,63.0,356.0,0.0,211.0,0.0,0.0,0.0,216.0,763.0,100.0,0.0,0.0


##3 Обогащение данных

И в train и в test не хватает существенной части данных. Заполним их средними значениями для каждого соответствующего модуля и презентации. Выберем использующиеся колонки:

In [ ]:
use_columns = ['code_module', 'code_presentation',
       'gender', 'region', 'highest_education', 'imd_band',
       'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability',
       'mat_seen_ratio_dataplus',
       'mat_seen_ratio_dualpane', 'mat_seen_ratio_externalquiz',
       'mat_seen_ratio_folder', 'mat_seen_ratio_forumng',
       'mat_seen_ratio_glossary', 'mat_seen_ratio_homepage',
       'mat_seen_ratio_htmlactivity', 'mat_seen_ratio_oucollaborate',
       'mat_seen_ratio_oucontent', 'mat_seen_ratio_ouelluminate',
       'mat_seen_ratio_ouwiki', 'mat_seen_ratio_page',
       'mat_seen_ratio_questionnaire', 'mat_seen_ratio_quiz',
       'mat_seen_ratio_resource', 'mat_seen_ratio_subpage',
       'mat_seen_ratio_url', 'mat_seen_ratio_sharedsubpage',
       'mat_seen_ratio_repeatactivity',
       'sum_click_dataplus',
       'sum_click_dualpane', 'sum_click_externalquiz', 'sum_click_folder',
       'sum_click_forumng', 'sum_click_glossary', 'sum_click_homepage',
       'sum_click_htmlactivity', 'sum_click_oucollaborate',
       'sum_click_oucontent', 'sum_click_ouelluminate', 'sum_click_ouwiki',
       'sum_click_page', 'sum_click_questionnaire', 'sum_click_quiz',
       'sum_click_resource', 'sum_click_subpage', 'sum_click_url',
       'sum_click_sharedsubpage', 'sum_click_repeatactivity']

In [ ]:
def fill(data):

  for col in use_columns[9:]:

    meta_condition = data[col].isna()
    codes = data[meta_condition][['code_module', 'code_presentation']].drop_duplicates()

    for code in codes.values:

      code_module = data['code_module'] == code[0]
      code_pres = data['code_presentation'] == code[1]

      value = round(data[code_module & code_pres][col].mean(), 2)

      data[col] = data[col].mask(code_module & code_pres & meta_condition, value)

  return data

In [ ]:
train = fill(train)
test = fill(test)

#3 CatBoost

##3.1 Базовая модель

Используем в качестве модели CatBoost.

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.5 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

Определим категориальные признаки:

In [ ]:
cat_col_train = np.where(train[use_columns].dtypes == np.object)[0]
cat_col_train

<ipython-input-135-453b1216f84e>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cat_col_train = np.where(train[use_columns].dtypes == np.object)[0]


array([0, 1, 2, 3, 4, 5, 6, 9])

In [ ]:
X = train[use_columns]
Y = train['final_result']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

Базовый результат:

In [ ]:
cbc = CatBoostClassifier(iterations=1000,
                          depth=8,
                          learning_rate=0.03,
                          loss_function='Logloss',
                          random_seed=42)

train_pool = Pool(X_train, Y_train, cat_features=cat_col_train)
val_pool = Pool(X_val, Y_val, cat_features=cat_col_train)
test_pool = Pool(X_val, cat_features=cat_col_train)

cbc.fit(train_pool, logging_level='Silent', eval_set=val_pool, early_stopping_rounds=20)

preds = cbc.predict(test_pool)

f1 = f1_score(Y_val, preds)

print("f1 on test set: {:.4f}".format(f1))

f1 on test set: 0.8995


##3.2 Подбор гиперпараметров

Подберем наилучшие гиперпараметры:

In [ ]:
max_f1 = 0
func_best = 0
n_best = 0
depth_best = 0

train_pool = Pool(X_train, Y_train, cat_features=cat_col_train)
val_pool = Pool(X_val, Y_val, cat_features=cat_col_train)
test_pool = Pool(X_val, cat_features=cat_col_train)

for func in ['Logloss']:
  for n in [300, 500, 800, 1000]:
    for depth in [4, 5, 8, 10, 12]:
      for rate in [0.03]:

        params = {'iterations': n,
                  'depth': depth,
                  'learning_rate': rate,
                  'objective': func,
                  'random_seed': 42}

        cbc = CatBoostClassifier(**params)

        cbc.fit(train_pool, logging_level='Silent', eval_set=val_pool, early_stopping_rounds=20)

        f1 = f1_score(Y_val, cbc.predict(test_pool))

        print('Текущий тест: n_estimators = ', n, 'max_depth = ', depth, 'loss_func = ', func, 'learning_rate = ', rate)
        print("f1 on test set: {:.4f}".format(f1))
        print('\n----------------')

        if f1 > max_f1:
          max_f1 = f1

          func_best = func
          n_best = n
          depth_best = depth

print('Лучший набор гиперпараметров: n_estimators = ', n_best, 'max_depth = ', depth_best, 'loss_func = ', func_best, 'learning_rate = ', rate)

Текущий тест: n_estimators =  300 max_depth =  4 loss_func =  Logloss learning_rate =  0.03
f1 on test set: 0.8881

----------------
Текущий тест: n_estimators =  300 max_depth =  5 loss_func =  Logloss learning_rate =  0.03
f1 on test set: 0.8885

----------------
Текущий тест: n_estimators =  300 max_depth =  8 loss_func =  Logloss learning_rate =  0.03
f1 on test set: 0.8957

----------------
Текущий тест: n_estimators =  300 max_depth =  10 loss_func =  Logloss learning_rate =  0.03
f1 on test set: 0.8957

----------------
Текущий тест: n_estimators =  300 max_depth =  12 loss_func =  Logloss learning_rate =  0.03
f1 on test set: 0.8940

----------------
Текущий тест: n_estimators =  500 max_depth =  4 loss_func =  Logloss learning_rate =  0.03
f1 on test set: 0.8909

----------------
Текущий тест: n_estimators =  500 max_depth =  5 loss_func =  Logloss learning_rate =  0.03
f1 on test set: 0.8934

----------------
Текущий тест: n_estimators =  500 max_depth =  8 loss_func =  Loglo

Обучим модель с лучшими гиперпараметрами:

In [ ]:
cbc = CatBoostClassifier(iterations=800,
                          depth=8,
                          learning_rate=0.03,
                          loss_function='Logloss',
                          random_seed=42)

train_pool = Pool(X_train, Y_train, cat_features=cat_col_train)
val_pool = Pool(X_val, Y_val, cat_features=cat_col_train)
test_pool = Pool(X_val, cat_features=cat_col_train)

cbc.fit(train_pool, logging_level='Silent', eval_set=val_pool, early_stopping_rounds=20)

preds = cbc.predict(test_pool)

f1 = f1_score(Y_val, preds)

print("f1 on test set: {:.4f}".format(f1))

f1 on test set: 0.8995


##3.3 Предсказание

In [ ]:
cat_col_sub = np.where(test[use_columns].dtypes == np.object)[0]
cat_col_sub

sub_pool = Pool(test[use_columns], cat_features=cat_col_sub)

sub_pred = cbc.predict(sub_pool)

sub_pred

submission = pd.read_csv('Submission_clf.csv')

submission.final_result = sub_pred
submission.to_csv('Submission.csv', index=False)

<ipython-input-129-ca6e5742d6dd>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cat_col_sub = np.where(test[use_columns].dtypes == np.object)[0]
